# Mini-Tarea 1: Análisis de datos de preferencias musicales

La presente actividad debe ser realizada de forma individual. El formato de entregar es el archivo .ipynb con todas las celdas ejecutadas. Las secciones donde se planteen preguntas de forma explícita, deben ser respondida en celdas de texto, y no se aceptará solo el output de una celda de código como respuesta.

**Fecha de entrega:** martes 13 de octubre de 2020, 09:00 hrs.

**Nombre alumno:** Darwin Joan Pérez Avendaño.

# Apache Pig

## Instalación de ambiente

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving hadoop_colab_installer.py to hadoop_colab_installer.py
User uploaded file "hadoop_colab_installer.py" with length 13493 bytes


In [2]:
exec(open('hadoop_colab_installer.py').read())

Active services:
2115 JobHistoryServer
1765 DataNode
2166 Jps
2057 NodeManager
1612 ResourceManager
1694 NameNode



## Descarga de dataset

In [3]:
!wget -q --show-progress http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-1K.tar.gz
!tar xzf lastfm-dataset-1K.tar.gz

lastfm-dataset-1K.t 100%[===================>] 641.58M  1.65MB/s    in 6m 34s  


## Actividad 0

In [ ]:
!hdfs dfs -mkdir lastfm_dir 

In [ ]:
!hdfs dfs -put lastfm-dataset-1K lastfm_dir

In [4]:
!hdfs dfs -mkdir hdfs_user_dir
!hdfs dfs -mkdir hdfs_profile_dir
!hdfs dfs -mkdir hdfs_output_dir

In [5]:
!hdfs dfs -put lastfm-dataset-1K/userid-profile.tsv hdfs_user_dir
!hdfs dfs -put lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv hdfs_profile_dir

In [6]:
!hdfs dfs -ls hdfs_user_dir
!hdfs dfs -ls hdfs_profile_dir
!hdfs dfs -ls hdfs_output_dir

Found 1 items
-rw-r--r--   1 root supergroup      37842 2020-10-13 03:15 hdfs_user_dir/userid-profile.tsv
Found 1 items
-rw-r--r--   1 root supergroup 2529193595 2020-10-13 03:15 hdfs_profile_dir/userid-timestamp-artid-artname-traid-traname.tsv


In [7]:
%%writefile my_pig_udfs.py

from pig_util import outputSchema
import array

@outputSchema('detected:boolean')
def empty_detector( gender ):
  if gender is None:
    return True
  query = gender.tostring() # 'array' to 'str'
  if len( query ) == 0:
    return True
  return False

Writing my_pig_udfs.py


## Actividad 1

In [8]:
%%writefile artista_mas_popular.pig

REGISTER 'my_pig_udfs.py' USING jython AS my_pig_udfs;

raw = LOAD 'hdfs_profile_dir/userid-timestamp-artid-artname-traid-traname.tsv' USING PigStorage('\t') AS (userid,  timestamp, musicbrainz_artist_id, artist_name, musicbrainz_track_id, track_name);

A = FOREACH raw GENERATE artist_name;

B = GROUP A BY artist_name;

C = FOREACH B GENERATE $0 AS new_name, COUNT($1) AS count_rep;

D = ORDER C BY count_rep DESC;

E = LIMIT D 10;

DUMP E;

STORE E INTO 'hdfs_output_dir/output/artista_mas_popular';

Writing artista_mas_popular.pig


In [9]:
!pig -f artista_mas_popular.pig


20/10/13 03:16:46 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
20/10/13 03:16:46 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
20/10/13 03:16:46 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2020-10-13 03:16:46,286 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2020-10-13 03:16:46,286 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1602559006282.log
2020-10-13 03:16:46,847 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2020-10-13 03:16:46,934 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2020-10-13 03:16:46,934 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: hdfs://localhost:9000
2020-10-13 03:16:47,511 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the session:

In [ ]:
!hdfs dfs -ls hdfs_output_dir/output/

In [14]:
!hdfs dfs -cat hdfs_output_dir/output/artista_mas_popular/part-r-00000

Radiohead	115209
The Beatles	100338
Nine Inch Nails	84421
Muse	63351
Coldplay	62251
Depeche Mode	59910
Pink Floyd	58561
Death Cab For Cutie	58083
Placebo	53543
Elliott Smith	50278


Responda la siguiente pregunta:

¿ Cuál es el artista más popular y el menos popular dele ranking top-10 ?

***R.*** El artista más popular del **Top-10** es **Radiohead** y el artista menos popular del **Top-10** es **Elliott Smith**.


## Actividad 2

In [10]:
%%writefile distribucion_por_genero.pig

REGISTER 'my_pig_udfs.py' USING jython AS my_pig_udfs;

raw = LOAD 'hdfs_profile_dir/userid-timestamp-artid-artname-traid-traname.tsv' USING PigStorage('\t') AS (userid,  timestamp, musicbrainz_artist_id, artist_name, musicbrainz_track_id, track_name);

raw_1 = LOAD 'hdfs_user_dir/userid-profile.tsv' USING PigStorage('\t') AS (userid, gender, age, country, registered);

A = FOREACH raw GENERATE userid, artist_name, musicbrainz_track_id;

B = FOREACH raw_1 GENERATE userid, gender;

C = JOIN A BY (userid),B BY (userid);

D = FILTER C BY artist_name == 'Radiohead' AND SIZE(gender) > 0;

E = GROUP D BY gender;

F = FOREACH E GENERATE $0 AS new_gender, COUNT($1) AS cant_rep;

DUMP F;

STORE F INTO 'hdfs_output_dir/output/distribucion_por_genero';

Writing distribucion_por_genero.pig


In [11]:
!pig -f distribucion_por_genero.pig


20/10/13 03:28:01 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
20/10/13 03:28:01 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
20/10/13 03:28:01 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2020-10-13 03:28:01,336 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2020-10-13 03:28:01,337 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1602559681334.log
2020-10-13 03:28:01,889 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2020-10-13 03:28:02,000 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2020-10-13 03:28:02,000 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: hdfs://localhost:9000
2020-10-13 03:28:02,577 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the session:

In [ ]:
!hdfs dfs -ls hdfs_output_dir/output/

In [15]:
!hdfs dfs -cat hdfs_output_dir/output/distribucion_por_genero/part-r-00000

f	43748
m	63784


Responda la siguiente pregunta:

¿ Cuál es la cantidad de hombres y mujeres que han escuchado alguna canción del artista más popular ?

***R.*** Un total de **43748 Mujeres** y un total de **63784 Hombres** han escuchado alguna canción de **Radiohead**.

## Actividad 3

In [12]:
%%writefile distribucion_por_edad.pig

REGISTER 'my_pig_udfs.py' USING jython AS my_pig_udfs;

raw = LOAD 'hdfs_profile_dir/userid-timestamp-artid-artname-traid-traname.tsv' USING PigStorage('\t') AS (userid,  timestamp, musicbrainz_artist_id, artist_name, musicbrainz_track_id, track_name);

raw_1 = LOAD 'hdfs_user_dir/userid-profile.tsv' USING PigStorage('\t') AS (userid, gender, age:int, country, registered);

A = FOREACH raw GENERATE userid, artist_name, musicbrainz_track_id;

B = FOREACH raw_1 GENERATE userid, age;

C = JOIN A BY (userid),B BY (userid);

D = FILTER C BY artist_name == 'Radiohead' AND age < 36;

E = GROUP D BY age;

F = FOREACH E GENERATE $0 AS new_age, COUNT($1) AS cant_rep;

G = ORDER F BY new_age ASC;

DUMP G;

STORE G INTO 'hdfs_output_dir/output/distribucion_por_edad';

Writing distribucion_por_edad.pig


In [13]:
!pig -f distribucion_por_edad.pig


20/10/13 03:34:55 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
20/10/13 03:34:55 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
20/10/13 03:34:55 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2020-10-13 03:34:55,562 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2020-10-13 03:34:55,562 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1602560095560.log
2020-10-13 03:34:56,217 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2020-10-13 03:34:56,352 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2020-10-13 03:34:56,352 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: hdfs://localhost:9000
2020-10-13 03:34:56,961 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the session:

In [ ]:
!hdfs dfs -ls hdfs_output_dir/output/

In [17]:
!hdfs dfs -cat hdfs_output_dir/output/distribucion_por_edad/part-r-00000

3	2
4	14
7	2
15	2
16	122
17	444
18	65
19	4514
20	5760
21	1589
22	7038
23	2552
24	1313
25	1289
26	2527
27	2360
28	1330
29	359
30	617
31	146
32	433
33	1441
34	404
35	1543


Responda la siguiente pregunta:

¿ Cuántos usuarios de 35 años han escuchado al artista más popular ?

***R.*** **1543** usuarios de una edad de 35 años han escuchado **Radiohead**.